In [35]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
import re 
import json
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from scipy import stats

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



In [36]:
def preprocess(data):
    # Replace single quotes with double quotes in "random load mesures" column
    data["random load mesures"] = data["random load mesures"].str.replace("'",'"')
    data['building'] = le.fit_transform(data['building'])
    data['Town'] = le.fit_transform(data['Town'])
    data['Total Wall Area'] = data['Wall Rt'] * data['Height'] * data['Number of Floors']
    #data['Total Window Area'] = data['Windows Rt'] * data['Height'] * data['WWR'] * data['Number of Floors']
    data['Total Floor Area'] = data['Ground Floor Rt'] * data['Height']
    #data['Total Roof Area'] = data['Roof Rt'] * data['Height']
    data['Total Internal Wall Area'] = data['Internal Wall Rt'] * data['Height'] * data['Number of Floors']
    data['Total Internal Floor Area'] = data['Internal Floor Rt'] * data['Height']
    data['Overall Energy Consumption'] = data['Operating Hours'] * data['EUI']

    
    pattern = r'\d+\.\d+'
    wwr = data['WWR'].apply(lambda x: max([float(m) for m in re.findall(pattern, x)]))
    data['WWR'] = wwr
    # assuming 'File' contains unique file names for each building shape, it can be removed
    
    
    features = data['random load mesures'].apply(lambda x: json.loads(x))
    cooling = features.apply(lambda x: float(x['Cooling'].replace(":C","")))
    lights = features.apply(lambda x: float(x['Lights'].replace(":C","")))
    data['Coolings'] = cooling
    data['Lights'] = lights
    
    #data['Lighting Impact on Heat Load'] = data['Lights'] * data['Light Heat Gain']
    data['Energy Efficiency'] = data['EUI'] / data['Number of Floors']
    data['Lighting Heat Addition Rate'] = data['Light Heat Gain'] / data['Operating Hours']
    

    scaler = StandardScaler()
    data[['Cooling Setpoint','Coolings','Lights', 'EUI', 'Cooling COP', 'Operating Hours', 'WWR',      'Equipment Heat Gain', 'Internal Wall Rt', 'Internal Floor Rt', 'Infiltration','Ground Floor Rt', 'Number of Floors', 'Occupancy', 'Light Heat Gain', 'Windows Rt',      'Height', 'Heating COP', 'Heating Setpoint', 'Wall Rt', 'Start Time', 'windows g-value',      'Roof Rt', 'Boiler Efficiency', 'Internal Mass', 'Permeability', 'Total Floors Area',]] = scaler.fit_transform(data[['Cooling Setpoint','Coolings','Lights', 'EUI', 'Cooling COP', 'Operating Hours', 'WWR',                                                              'Equipment Heat Gain', 'Internal Wall Rt', 'Internal Floor Rt', 'Infiltration',                                                              'Ground Floor Rt', 'Number of Floors', 'Occupancy', 'Light Heat Gain', 'Windows Rt',                                                              'Height', 'Heating COP', 'Heating Setpoint', 'Wall Rt', 'Start Time', 'windows g-value',                                                              'Roof Rt', 'Boiler Efficiency', 'Internal Mass', 'Permeability', 'Total Floors Area']])
    """
    interaction_features = ['Overall Energy Consumption', 'Lighting Impact on Heat Load', 'Energy Efficiency','Lighting Heat Addition Rate']
    for feature1 in interaction_features:
        for feature2 in interaction_features:
            if feature1 != feature2:
                interaction_feature_name = f'{feature1} x {feature2}'
                data[interaction_feature_name] = data[feature1] * data[feature2]"""

    # assuming we want to select the top 10 features with the highest F-test score
    selector = SelectKBest(f_regression, k=10)
    data = data.drop(['random load mesures', 'Permeability','File','building'], axis=1)
    #data = selector.fit_transform(data.drop('Operational Energy', axis=1), data['Operational Energy'])
    # assume your data is stored in a NumPy array called `data`
    for col in data.columns:
        if col == 'Operational Energy':
            continue  # skip target variable
        median = np.median(data[col])
        q1 = np.percentile(data[col], 25)
        q3 = np.percentile(data[col], 75)
        iqr = q3 - q1
        upper_bound = q3 + 1.5 * iqr
        lower_bound = q1 - 1.5 * iqr
        data.loc[data[col] > upper_bound, col] = median
        data.loc[data[col] < lower_bound, col] = median

    return data


In [37]:
train = pd.read_csv("/kaggle/input/traintrain/Train.csv")
test = pd.read_csv("/kaggle/input/testing/Test (1).csv")

# Preprocess train and test data
train = preprocess(train)
test = preprocess(test)



In [38]:
corr = train.corr()
corr.style.background_gradient(cmap='coolwarm')

,Cooling Setpoint,EUI,Cooling COP,Operating Hours,WWR,Equipment Heat Gain,Internal Wall Rt,Internal Floor Rt,Infiltration,Ground Floor Rt,Number of Floors,Occupancy,Light Heat Gain,Windows Rt,Height,Heating COP,Heating Setpoint,Town,Wall Rt,Start Time,windows g-value,Roof Rt,Boiler Efficiency,Internal Mass,Total Floors Area,Operational Energy,Total Wall Area,Total Floor Area,Total Internal Wall Area,Total Internal Floor Area,Overall Energy Consumption,Coolings,Lights,Energy Efficiency,Lighting Heat Addition Rate
Cooling Setpoint,1.000000,-0.068107,0.000361,-0.002512,0.000626,-0.000513,0.000218,-0.000725,-0.000061,0.000391,-0.007810,-0.000890,0.000606,-0.001047,-0.001023,-0.000509,0.000329,-0.000000,0.000812,-0.000069,0.001506,-0.000691,0.002271,0.000778,-0.000180,-0.039054,-0.002268,0.000124,0.000415,-0.000793,-0.052219,-0.183700,0.001695,-0.028094,0.001885
EUI,-0.068107,1.000000,-0.100499,0.226640,0.065295,-0.216274,0.000674,-0.000881,0.431350,-0.027666,-0.137241,-0.143993,0.243494,-0.080551,0.224926,-0.013619,0.239733,-0.007497,-0.059645,0.004305,-0.053063,-0.045107,-0.034087,0.009673,-0.296377,0.223445,-0.143103,0.054075,-0.113964,0.104565,0.821753,-0.053427,-0.081159,0.590876,0.075869
Cooling COP,0.000361,-0.100499,1.000000,-0.004606,0.000960,-0.000764,-0.001146,0.000083,0.001606,0.000707,0.000680,-0.000827,-0.001208,-0.001157,0.000436,0.003137,0.000613,-0.000000,0.000125,-0.000148,0.000746,0.003228,0.001324,0.001223,0.000643,-0.059204,0.002288,0.000733,0.003904,0.000271,-0.076724,-0.001305,-0.002114,-0.054451,0.001683
Operating Hours,-0.002512,0.226640,-0.004606,1.000000,-0.002839,0.002102,0.000105,0.001324,-0.001730,0.002081,0.002429,0.001964,-0.000973,0.000647,-0.001629,-0.001973,-0.001286,0.000000,-0.000421,0.051274,-0.001301,-0.000124,0.003918,0.001420,0.001527,0.137906,-0.002814,0.001317,0.000963,0.000681,0.674695,0.156523,0.237029,0.115028,-0.529675
WWR,0.000626,0.065295,0.000960,-0.002839,1.000000,-0.001507,0.001600,0.002230,-0.002454,-0.001885,-0.000899,-0.000996,0.003243,0.000464,-0.002380,-0.003404,0.000252,-0.000000,-0.000126,0.002118,0.002503,0.001921,0.000813,0.003471,0.001442,0.039783,-0.000250,-0.002250,0.001599,0.001460,0.045600,0.242510,-0.018727,0.037111,0.005376
Equipment Heat Gain,-0.000513,-0.216274,-0.000764,0.002102,-0.001507,1.000000,0.001937,-0.001850,-0.000544,0.000229,-0.000199,-0.000633,-0.001371,-0.000932,-0.001507,-0.000401,0.002835,-0.000000,0.000449,-0.000144,-0.000576,-0.001928,0.000576,-0.000421,0.001083,-0.127682,0.003082,0.000740,0.002038,-0.002446,-0.152739,0.190053,0.004531,-0.108204,-0.000598
Internal Wall Rt,0.000218,0.000674,-0.001146,0.000105,0.001600,0.001937,1.000000,0.000413,0.000928,-0.000076,-0.000822,-0.000480,0.000088,-0.001163,-0.001437,0.000224,-0.001981,0.000000,0.001851,-0.000432,-0.001709,0.000136,0.001533,0.000126,0.001692,0.003001,0.003483,-0.000048,0.514970,-0.000827,0.002638,0.006048,0.001862,0.001506,-0.000110
Internal Floor Rt,-0.000725,-0.000881,0.000083,0.001324,0.002230,-0.001850,0.000413,1.000000,-0.000098,0.000044,-0.000770,0.001380,-0.000924,-0.001945,-0.000184,-0.000803,-0.001554,0.000000,0.000008,-0.001275,0.000219,0.001181,-0.006593,0.002320,-0.001584,-0.002452,-0.002939,-0.000195,-0.001674,0.876969,0.002622,0.001421,-0.001489,0.000594,-0.001932
Infiltration,-0.000061,0.431350,0.001606,-0.001730,-0.002454,-0.000544,0.000928,-0.000098,1.000000,0.002599,-0.241279,-0.004151,-0.002807,-0.003537,-0.110880,-0.001455,-0.000518,0.000000,0.003504,-0.001737,0.000406,0.001178,-0.002178,-0.001709,-0.405366,-0.173383,-0.262293,-0.036796,-0.298077,-0.051777,0.305047,-0.353930,-0.319835,0.387501,-0.002124
Ground Floor Rt,0.000391,-0.027666,0.000707,0.002081,-0.001885,0.000229,-0.000076,0.000044,0.002599,1.000000,-0.001426,0.001735,-0.000783,0.000336,-0.000209,0.001266,0.001886,-0.000000,0.000376,-0.000132,-0.000123,0.002631,0.001936,0.000687,-0.000359,-0.016440,0.002464,0.922979,0.000283,-0.000136,-0.020094,0.024900,0.001910,-0.010855,-0.001796


In [39]:
X = train.drop(['Operational Energy'], axis=1)
y = train['Operational Energy']
n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

# Initialize the list to store the mean squared errors (MSEs)
mse_list = []

In [40]:
from sklearn.ensemble import GradientBoostingRegressor
for train_index, val_index in kf.split(X):
    
    # Split the data into training and validation sets
    X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]
    
    # Scale the data using StandardScaler
    scaler = StandardScaler()
    #X_train_fold_scaled = scaler.fit_transform(X_train_fold)
    #X_val_fold_scaled = scaler.transform(X_val_fold)
    
    # Apply PCA to reduce dimensionality
    pca = PCA(n_components=10)
    X_train_fold_pca = pca.fit_transform(X_train_fold)
    X_val_fold_pca = pca.transform(X_val_fold)
    
    # Train the model
    dt = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
    dt.fit(X_train_fold_pca, y_train_fold)
    
    # Make predictions on the validation set and calculate MSE
    y_pred = dt.predict(X_val_fold_pca)
    mse = mean_squared_error(y_val_fold, y_pred)
    # Append the MSE to the list
    mse_list.append(mse)
    
# Print the mean of the MSEs
print("Mean MSE:", np.mean(mse_list))

Mean MSE: 1379948334.8729603


In [41]:
y_nnn=scaler.fit_transform(test)
y_nnn=pca.transform(y_nnn)
y_nnn = dt.predict(y_nnn)
y_nnn

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


array([59869.17100637, 60748.16029199, 50826.24901896, ...,
       76893.29169062, 60786.35471754, 66846.42188916])

In [42]:
importances = dt.feature_importances_

# Sort feature importances in descending order
sorted_importances = sorted(zip(importances, X.columns), reverse=True)

# Print the feature importances
for importance, feature in sorted_importances:
    print(f"{feature}: {importance}")


Internal Wall Rt: 0.44082546178782667
EUI: 0.28462245717088225
Cooling Setpoint: 0.09748670519386038
Cooling COP: 0.09215223102062368
Infiltration: 0.056505528034376036
Internal Floor Rt: 0.017816259222224413
WWR: 0.0063750883274671915
Operating Hours: 0.0029750422218512177
Equipment Heat Gain: 0.0006948490297240218
Ground Floor Rt: 0.0005463779911640479


In [43]:
importance_scores = dt.feature_importances_
feature_importances = dict(zip(X.columns, importance_scores))
top_5_features = sorted(feature_importances, key=feature_importances.get, reverse=True)[:7]

# Select only the top 5 features
X = X[top_5_features]
test = test[top_5_features]

# Set up the KFold object
n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

# Initialize the list to store the mean squared errors (MSEs)
mse_list = []

# Loop over the folds
for train_index, val_index in kf.split(X):
    
    # Split the data into training and validation sets
    X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]

    # Scale the features
    scaler = StandardScaler()
    #X_train_fold = scaler.fit_transform(X_train_fold)
    #X_val_fold = scaler.transform(X_val_fold)
    
    # Reduce dimensionality using PCA
    pca = PCA(n_components=2)
    X_train_fold = pca.fit_transform(X_train_fold)
    X_val_fold = pca.transform(X_val_fold)

    # Train the model and predict on the validation set
    dt.fit(X_train_fold, y_train_fold)
    y_pred = dt.predict(X_val_fold)

    # Compute the mean squared error
    mse = mean_squared_error(y_val_fold, y_pred)
    mse_list.append(mse)

# Compute the average mean squared error over the folds
avg_mse = np.mean(mse_list)
print("Average MSE:", avg_mse)
#Average MSE: 4133652408.6541567


Average MSE: 4103512503.4575615


In [44]:
y_4bb=scaler.fit_transform(test)
y_4bb=pca.transform(y_4bb)
y_4bb = dt.predict(y_4bb)
y_4bb

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


array([162881.83003675, 161611.49608111, 165892.66064819, ...,
       143072.66899438, 172387.04272827, 139679.88718778])

In [45]:
csv_file=pd.read_csv("/kaggle/input/simple/SampleSubmission.csv")
csv_file['Operational Energy']=y_4bb
csv_file.to_csv("top___000.csv",index=False)